In [ ]:
import mne
import numpy as np
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:

att_file = r"C:\Users\LENOVO\OneDrive - City University\Desktop\Attention detection FYP\S01\S01_clean\Att_S01_cleaned_raw3.fif"
inatt_file = r"C:\Users\LENOVO\OneDrive - City University\Desktop\Attention detection FYP\S01\S01_clean\Inatt_S01_cleaned_raw3.fif"
# Load the cleaned data (Raw or Epochs)
att_raw = mne.io.read_raw_fif(att_file, preload=True)
inatt_raw = mne.io.read_raw_fif(inatt_file, preload=True)

# Define frequency bands
theta_band = (4, 8)
beta_band = (13, 30)
# alpha_band = (8, 13)

def compute_theta_beta_ratio(raw, epoch_length=4.0):
    sfreq = raw.info['sfreq']
    n_samples = int(epoch_length * sfreq)
    data = raw.get_data(picks='Fp1')
    
    ratios = []
    for start in range(0, data.shape[1] - n_samples, n_samples):
        epoch = data[:, start:start + n_samples]
        
        # Adjust n_fft to be <= n_samples to avoid the error
        n_fft = min(2048, epoch.shape[1])
        
        psd, freqs = mne.time_frequency.psd_array_welch(
            epoch, sfreq=sfreq, fmin=1, fmax=40, n_fft=n_fft, n_per_seg=n_fft
        )
        
        psd = np.mean(psd, axis=0)
        theta_power = psd[(freqs >= 4) & (freqs <= 8)].mean()
        beta_power = psd[(freqs >= 13) & (freqs <= 30)].mean()
        
        ratio = theta_power / beta_power if beta_power > 0 else np.nan
        ratios.append(ratio)
    
    return np.array(ratios)


# Calculate ratios for attention and inattention
att_ratios = compute_theta_beta_ratio(att_raw)
inatt_ratios = compute_theta_beta_ratio(inatt_raw)
print(f"Attention ratios: {att_ratios}")
print(f"Inattention ratios: {inatt_ratios}")
# Clean NaNs
att_ratios = np.nan_to_num(att_ratios, nan=1e-10)
inatt_ratios = np.nan_to_num(inatt_ratios, nan=1e-10)

# Run paired t-test (make sure arrays are same length)
min_len = min(len(att_ratios), len(inatt_ratios))
t_stat, p_val = ttest_rel(att_ratios[:min_len], inatt_ratios[:min_len])

print(f"Paired t-test: t({min_len-1}) = {t_stat:.3f}, p = {p_val:.4f}")


In [ ]:

# Define file paths
att_file = r"C:\Users\LENOVO\OneDrive - City University\Desktop\Attention detection FYP\S05\S05_clean\Att_S05_cleaned_raw2.fif"
inatt_file = r"C:\Users\LENOVO\OneDrive - City University\Desktop\Attention detection FYP\S05\S05_clean\Inatt_S05_cleaned_raw2.fif"
specified_channels = ['Fp1', 'Fp2', 'Fz', 'F3', 'F4', 'C3', 'Cz', 'C4']

# Load EEG data
# try:
att_raw = mne.io.read_raw_fif(att_file, preload=True, verbose=False)
inatt_raw = mne.io.read_raw_fif(inatt_file, preload=True, verbose=False)
print("Successfully loaded EEG files.")

for ch_list, label, raw in zip([specified_channels]*2, ['Attention', 'Inattention'], [att_raw, inatt_raw]):
        missing = [ch for ch in ch_list if ch not in raw.ch_names]
        if missing:
            raise ValueError(f"Missing channels in {label} data: {missing}")



# Define frequency bands
theta_band = (4, 8)
beta_band = (13, 30)

def compute_theta_beta_ratios_per_channel_epoch(raw, epoch_length=10.0, picks=None):
    sfreq = raw.info['sfreq']
    n_samples = int(epoch_length * sfreq)
    if picks is None:
        picks_indices = mne.pick_types(raw.info, eeg=True, exclude='bads')
        ch_names = [raw.ch_names[i] for i in picks_indices]
    else:
        ch_names = picks
        picks_indices = mne.pick_channels(raw.ch_names, ch_names, ordered=True)

    if len(picks_indices) == 0:
        print("No valid channels selected.")
        return np.array([]), []

    data = raw.get_data(picks=picks_indices)
    all_epochs_channel_ratios = []
    if data.shape[1] < n_samples:
        print("Not enough data for even one epoch.")
        return np.array([]), ch_names

    for start in range(0, data.shape[1] - n_samples + 1, n_samples):
        epoch_data = data[:, start:start + n_samples]
        n_fft = min(1500, epoch_data.shape[1])
        psd, freqs = mne.time_frequency.psd_array_welch(epoch_data, sfreq=sfreq, fmin=1, fmax=40, n_fft=n_fft, n_per_seg=n_fft, verbose=False)
        theta_mask = (freqs >= theta_band[0]) & (freqs <= theta_band[1])
        beta_mask = (freqs >= beta_band[0]) & (freqs <= beta_band[1])

        if not np.any(theta_mask) or not np.any(beta_mask):
            all_epochs_channel_ratios.append([np.nan] * len(ch_names))
            continue

        theta_power = psd[:, theta_mask].mean(axis=1)
        beta_power = psd[:, beta_mask].mean(axis=1)
        ratios = [t / b if b > 0 else np.nan for t, b in zip(theta_power, beta_power)]
        all_epochs_channel_ratios.append(ratios)

    return np.array(all_epochs_channel_ratios), ch_names

# Compute ratios
att_ratios_2d, ch_names = compute_theta_beta_ratios_per_channel_epoch(att_raw, picks=specified_channels)
inatt_ratios_2d, _ = compute_theta_beta_ratios_per_channel_epoch(inatt_raw, picks=specified_channels)

# Average over epochs per channel
att_mean_ratios = np.nanmean(att_ratios_2d, axis=0)
inatt_mean_ratios = np.nanmean(inatt_ratios_2d, axis=0)

print("\nAverage Theta/Beta Ratios:")
for ch, a, i in zip(ch_names, att_mean_ratios, inatt_mean_ratios):
    print(f"  {ch}: Attention = {a:.4f}, Inattention = {i:.4f}")

# Paired t-test across channels
if len(att_mean_ratios) == len(inatt_mean_ratios):
    t_stat, p_val = ttest_rel(att_mean_ratios, inatt_mean_ratios)
    print(f"\nPaired t-test on average ratios across channels: t({len(ch_names)-1}) = {t_stat:.3f}, p = {p_val:.4f}")
else:
    print("Cannot perform paired t-test: mismatched lengths.")

# Plot average bar chart
plt.figure(figsize=(max(8, len(ch_names) * 0.6), 6))
x = np.arange(len(ch_names))
width = 0.35
plt.bar(x - width/2, att_mean_ratios, width, label='Attention', color='skyblue')
plt.bar(x + width/2, inatt_mean_ratios, width, label='Inattention', color='salmon')
plt.ylabel('Mean Theta/Beta Ratio')
plt.title('Mean Theta/Beta Power Ratio per Channel')
plt.xticks(x, ch_names, rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
